# Core Data Engineering Principles

### Idempotence

Simply put, this is the idea that running a task in your dataflow with the same input more than once will have the same effect as running it exactly once.

Why would you want this? Say a later stage of your ETL process breaks so you have to rerun the whole thing and earlier stages that did run successfully are run again. Or the pipeline is triggered twice with the same input for some reason. If the jobs are idempotent you won't get nasty side effects, such as primary key errors throwing errors the second time around, or ending up with duplicated entries.

Practically, this could mean using upserts instead inserts, or checking to see if the stage has been applied on an input already, so that if it has, it won't run again.

### DAGs

Directed Acyclic Graphs. This is the idea that you structure your dataflows as a series of tasks, each of which has one concern. On top of that, your task scheduler is aware of dependencies between them. So a later stage of the pipeline will only kick off once all of the input dependencies for that task are met. This graph of tasks is directed because the dependency relation is one way, and acyclic because the there are no cycles in the graph of dependencies. It's a useful abstraction to be able to map out your tasks in terms of, for example: A relies on B, which itself relies of C & D.

### Results Caching

Relating to both of the above, a really useful idea is that of serializing your intermediate steps. By that I mean saving the results of each stage of your pipeline. You can use it enforce idempotence. ie. when processing an input, you can see if there already exists an output for that input, ala memoization. In those cases, you can shortcut to just returning the precomputed result, rather than reprocessing the input file. This is also faster, and allows for efficient rerunning of your pipeline if a later stage breaks. Finally, you can also inspect the contents of intermediate results to check the health of your pipeline and look for processing errors.

### ETL "as code"

This is an idea that Maxime Beauchemin writes about in his [article](https://www.freecodecamp.org/news/the-rise-of-the-data-engineer-91be18f1e603/), The Rise of the Data Engineer. He argues that the logic required to build modern data pipelines is too complex for drag-and-drop ETL tools, and that complexity is best represented in code.

Another advantage of ETL/Infrastructure as code, however, is ability to version control it, audit it and deploy it automatically.

# Data Modeling

### 3 types of data models:

- Conceptual data model (business stakeholders and data architects) -"What"
- Logical data model (data architects) - "How"
- Physical data model (DBA) - "How"

**Resources:**<br>
- [10-minute guide to data modeling](https://www.credera.com/blog/technology-solutions/data-modeling-explained-in-10-minutes-or-less/)
- [Data Modeling 101](http://www.agiledata.org/essays/dataModeling101.html)

# Normalization versus De-Normalization

- Normalization: breaking a table down into more tables to reduce redundancy and maintain data integrity
- De-Normalization: not breaking a table into smaller tables, consolidating to fewer tables, risk for data redundancy

### Resources:<br>
- [What is normalization?](https://www.guru99.com/database-normalization.html)

# The 3 Normal Forms or 3NF

- 1st Normal Form: Remove duplicate data and break data down to granular or more atomic level
- 2nd Normal Form: All columns in a table must depend on the primary key column
- 3rd Normal Form: A column should not be duplicated or exists in multiple tables

There are higher normal forms, but achieving 3NF is "good enough" for most real world cases.

### Not in 3NF:

![alt text](images/Poor_Design.png "Bad Design")

### In 3NF:

![alt text](images/Better_Design.png "Bad Design")

# Star vs Snowflake Schema

References:
- [Star vs Snowflake Schema](https://www.vertabelo.com/blog/technical-articles/data-warehouse-modeling-star-schema-vs-snowflake-schema)

### Star Schema Summary:

![alt text](images/Star_Schema_Summary.png "Star Schema Summary")

### Star Schema Example:

![alt text](images/Star_Schema.png "Star Schema")

### Snowflake Schema Summary:

![alt text](images/Snow_Flake_Summary.png "Snow Flake Summary")

### Snowflake Schema Example:

![alt text](images/Snow_Flake_Schema.png "Snow Flake Schema")

# YouTube Videos

- [Designing a traditional relational database](https://www.youtube.com/watch?v=I_rxqSJAj6U)
- [Data Warehouse Design](https://www.youtube.com/watch?v=--OJpdPeH80)
- [ETL Design](https://youtu.be/sLhInuwdwcc)
- [Star Schema vs Snowflake](https://youtu.be/Qq4yhhAk9fc)
- [OLAP vs OLTP](https://youtu.be/AiZWeSUjylU)
- [Slowly Changing Dimensions](https://youtu.be/1FZ7et0pN4c)

# Architectural Considerations

- [Architectural Considerations](https://medium.com/ssense-tech/principled-data-engineering-part-i-architectural-overview-6d4bdf89b657)
- Data Mart
- Database
- Data Warehouse
- Data Lake
    - **Raw Data:** A “raw data” bucket which contains the raw,dumps untransformed, and lossless incoming data from all our sources such as event streams from microservices, transactional database snapshots, log dumps, and data from third party sources via FTP or API calls. Data here can be in various formats such as CSV, JSON, flat text files etc., and may not have defined schemas. This bucket plays the key role of guaranteeing no data loss and replayability of our pipelines. In other words, if our pipelines downstream change or fail, our raw data store guarantees that all the original source data remains intact and available for re-processing.
    - **Interim Data:** An “interim data” bucket which imports the aforementioned raw data and performs the most minimal transformations required to homogenize its structure, impose schemas, and allow cataloging. Recommend using a format that enforces a schema like parquet. This eliminates a lot of dangerous ambiguities of schema-less data, which can lead to data loss and poor governance. The minimal transformations performed at this step also allow for some critical type management such as homogenizing date formats and number types (decimals, floats, doubles, etc.), and handling null values.
    - **Business Data:** A “business data” bucket which presents transformed datasets to end-users. Data here conforms to semantically meaningful naming conventions and each dataset corresponds to a specific business need. Furthermore, the datasets here have more refined schemas that make sense to our end-users — the consumers of this data.
- Pipeline
    - Desired characteristics:
        - Replayable or reproducible
        - Has idempotency: applied several times without changing the result beyond the initial application to prevent duplicate or corrupt data
    - Types of pipelines:
        - ETL: In an ETL pipeline, data is extracted from a source, transformed to the required shape, and inserted into the target. The advantage of ETL is that data enters your system in the shape you want it to be in, and can easily be modeled for analytics. This works exceedingly well when the data is coming from consistent and trusted sources, but can quickly become too brittle in cases where there is a possibility of a schema change, or the data cannot be re-extracted. Imagine a scenario wherein data is extracted from a third-party API, transformed and then loaded into a data warehouse. Some time later the process needs to be rerun, but the source data is no longer available due to the third-party aggregating its data after a certain amount of time (to reduce storage fees). In other words, your system does not offer immutable data.
        - ELT: ELT addresses this by extracting and loading the raw data immediately into storage. From there, you can rerun the transformation portion of the pipelines to your heart’s content. The drawback here is that the raw data can potentially be schemaless and unstructured. Managing this raw data becomes the main difficulty in this configuration, and in an era of ever increasing compliance, proper cataloging is critical.
        - Pipeline "as code" (use of programming languages)
        - GUI pipeline tools (Informatica, Talend, Pentaho, etc)

# Data Governance

- [Data Governance](https://medium.com/ssense-tech/principled-data-engineering-part-ii-data-governance-30297abb2446)
- Metadata Management
    - Data Cataloging: the "what" of your data: table names, column names, data type, etc
    - Data Lineage: the "where" of your data: what are the sources of your data

# OLAP - Online Analytical Processing System

- Typical of STAR Schema
- De-normalized (dimension and fact design)
- Faster analysis and search by combining tables
- Requires more data storage due to redundancy
- Less complex join complexity compared to OLTP

# OLTP - Online Transaction Processing System

- Typical of Snow Flake Schema
- Normalized (1st to 3rd normal form)
- Faster inserts, updates, deletes, and improved data quality due to reduced redundancy
- Requires less space due to reduction of redundancy
- Query performance slower compared to OLAP or STAR Schema due to more tables to scan
- Join complexity increases

# Hadoop Ecosystem - "Old" Big Data

**The 3 Main Layers**
- **HDFS:** Hadoop Distributed File System (storage layer) 
- **Yarn:** Yet Another Resource Negotiator: How to get all these computers to work together in an 
  orderly fashion (orchestration layer)
- **MapReduce:** How to do stuff with the distributed data in a cluster (execution layer / Java code)


# Apache Spark Ecosystem

- **Storage layer:** RDD, Apache Arrow
- **Query / ETL layer:** Spark SQL, Spark Dataframe, Apache Beam, Apache Pig, Apache Hive

# Amazon Web Services (AWS)

- **Storage layer:** S3 buckets
- **Query / ETL layer:** Athena (SQL syntax)

# Google Cloud Platform

To be added

# Tools

- ETL tools
    - AWS Glue
    - Apache Beam
    - Apache Spark
    - Dask
    - Apache Airflow (Python)
    - Prefect (Python)
    - Papermill (Python)
    - SAS
    - IBM Informatica
    - Talend
    - Pentaho
- SQL-like tools for Big Data
    - AWS Athena / PyAthena
    - Apache Spark SQL
    - Apache Hive
- Dataframe-like tools for Big Data
    - Spark dataframe
    - [Koalas](https://github.com/databricks/koalas)
    - Dask dataframe

# Data or Storage Formats

Schema-less formats
    - HDFS
    - S3
    - RDD
Schema formats
    - parquet (column oriented, on-disc)
    - avro (row oriented, on-disc)
    - orc (column oriented, on-disc)
    - arrow (column oriented, main memory store)

# Additional Learning Resources

- [Top 10 blogs about transitioning to data engineering](https://blog.insightdatascience.com/top-10-blog-posts-to-help-you-transition-to-data-engineering-1db2312ecdaf)